# Market-Basket Analysis

In this notebook we perform Market-Basket analysis by using the Python API **Machine Learning extensions or Mlxtend**.
http://rasbt.github.io/mlxtend/

More specifically, we perform the following two tasks using this API.
- Frequent itemsets generation
- Minining association rules

The API uses the **Apriori** algorithm to generate frequent itemsets.


## Transaction Data Format

The Market-Basket data is typically given using a Python **list of lists** structure.

For the purpose of programmatic processing by the standard APIs (e.g., Mlxtend), the Market-Basket data needs to be stored in a **binary format**. 

More specifically, in a transaction an item can be treated as a binary variable whose value is 1 if the item is present in a transaction and 0 otherwise. This is known as **one-hot encoding**.



## Input Data

The transaction data is stored in a CSV file.

Each row contains a comma separated list of items.

Thus, to create one-hot encoded data we do the following:
- Read from the CSV file as a python **list of lists**. Remove white spaces (empty items).
- One-hot encode the data from list of lists. 


### Create One-Hot Encoded Data
To create a one-hot encoded NumPy boolean array of the transaction data from a Python list of lists, we use the **TransactionEncoder** object from the mlextend API.

The TransactionEncoder uses its "fit" method to learn the unique labels in the dataset, and the "transform" method to transform the input dataset (a Python list of lists) into a one-hot encoded NumPy boolean array.

In [1]:
from time import time

import numpy as np
import pandas as pd
import csv

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## Helper Functions for Creating One-Hot Encoded DataFrame Object

We use following functions to create one-hot encoded DataFrame object from the transaction data stored in a CSV file.

- isInList(): Function to determine whether an item/itemset is contained in a list/basket.

- removeEmptyValues(): Function to remove strings (e.g., white space) from the list items.

- generateListOfListsFromCSV(): Function to create a **list of lists** from a CSV file containing transaction data.

- oneHotEncodedDataFrame(): Function to create a one-hot encoded DataFrame object from the Python **list of lists**. 

In [2]:
# Function to determine whether an item/itemset is contained in a list/basket
def isInList(aList, item):
    if item in aList:
        return 1
    else:
        return -1   

    
# Function to remove strings (e.g., white space) from the list items
def removeEmptyValues(aList, symbolToRemove):
    while(True):
        val = isInList(aList, symbolToRemove)
        if(val == 1):
            aList.remove(symbolToRemove)    
        else:
            break

        

# Function to create a list of lists from a CSV file containing transaction data
def generateListOfListsFromCSV(fullFileName):

    with open(fullFileName, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f, delimiter=",")
        data = list(reader)

    listOfLists = []

    for i in range(len(data)):
        removeEmptyValues(data[i], '')
        listOfLists.append(data[i])
        
    return listOfLists

 

# Function to create a one-hot encoded DataFrame object from the Python list of lists        
def oneHotEncodedDataFrame(data):

    # Transform the transaction data into a one-hot encoded NumPy boolean array 
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)


    # Read the one-hot encoded data as a Pandas DataFrame object
    dataFrame = pd.DataFrame(te_ary, columns=te.columns_)
    
    return dataFrame
        

## Load Data From CSV File and Create One-Hot Encoded DataFrame Object

In [3]:
# Load the CSV file transaction data as a **list of lists**.
dataset = generateListOfListsFromCSV('/Users/hasan/datasets/transactionsmall.csv')

print("\nDataset (list of lists):")
for i in range(len(dataset)):
    print(dataset[i])

# Convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object.
dataset_onehotencoded = oneHotEncodedDataFrame(dataset)

print("\nOne-hot Encoded Dataset:")
print(dataset_onehotencoded)


Dataset (list of lists):
['M1', 'M2', 'M3']
['M4', 'M5', 'M6']
['M1', 'M4']
['M1', 'M2', 'M4', 'M5']
['M1', 'M2', 'M3', 'M4']
['M1', 'M3']
['M2', 'M4', 'M5']
['M1', 'M2', 'M4', 'M5', 'M6']

One-hot Encoded Dataset:
      M1     M2     M3     M4     M5     M6
0   True   True   True  False  False  False
1  False  False  False   True   True   True
2   True  False  False   True  False  False
3   True   True  False   True   True  False
4   True   True   True   True  False  False
5   True  False   True  False  False  False
6  False   True  False   True   True  False
7   True   True  False   True   True   True


## Frequent Itemsets Generation

We use Mlxtend API's "apriori" function to generate frequent k-itemsets. This function requires the transaction data to be passed as a one-hot encoded DataFrame object.

        apriori(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0, low_memory=False)


### Parameters

- df : pandas DataFrame or pandas SparseDataFrame

        pandas DataFrame the encoded format. The allowed values are either 0/1 or True/False. 

- min_support : float (default: 0.5)

        A float between 0 and 1 for minumum support of the itemsets returned. The support is computed as the fraction transactions_where_item(s)_occur / total_transactions.

- use_colnames : bool (default: False)

        If True, uses the DataFrames' column names in the returned DataFrame instead of column indices.

- max_len : int (default: None)

        Maximum length of the itemsets generated. If None (default) all possible itemsets lengths (under the apriori condition) are evaluated.

- verbose : int (default: 0)

        Shows the number of iterations if >= 1 and low_memory is True. If=1 and low_memory is False, shows the number of combinations.

- low_memory : bool (default: False)

        If True, uses an iterator to search for combinations above min_support. Note that while low_memory=True should only be used for large dataset if memory resources are limited, because this implementation is approx. 3-6x slower than the default.


- Returns

pandas DataFrame with columns ['support', 'itemsets'] of all itemsets that are >= min_support and < than max_len (if max_len is not None). 

Each itemset in the 'itemsets' column is of type frozenset, which is a Python built-in type that behaves similarly to sets except that it is immutable (For more info, see https://docs.python.org/3.6/library/stdtypes.html#frozenset).

## Generate All Frequent Itemsets with at least 30% Support

In [4]:
t0 = time()   

frequent_itemsets = apriori(dataset_onehotencoded, min_support=0.3, use_colnames=True)

time_apriori = time() - t0

print("Frequent Itemsets Generation: Processing Time = %0.3fs\n" % time_apriori)

print(frequent_itemsets)

Frequent Itemsets Generation: Processing Time = 0.009s

    support      itemsets
0     0.750          (M1)
1     0.625          (M2)
2     0.375          (M3)
3     0.750          (M4)
4     0.500          (M5)
5     0.500      (M1, M2)
6     0.375      (M1, M3)
7     0.500      (M1, M4)
8     0.500      (M4, M2)
9     0.375      (M5, M2)
10    0.500      (M5, M4)
11    0.375  (M1, M4, M2)
12    0.375  (M4, M5, M2)


## Frequent Itemsets: Selecting and Filtering Results

The above code generates ALL frequent itemsets.

We can select only the desired itemsets of a **specific length** as well (e.g., frequent 2-itemsets).

To do this we add a new column to the "frequent_itemsets" DataFrame that stores the length of each itemset.

In [5]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

print(frequent_itemsets)

    support      itemsets  length
0     0.750          (M1)       1
1     0.625          (M2)       1
2     0.375          (M3)       1
3     0.750          (M4)       1
4     0.500          (M5)       1
5     0.500      (M1, M2)       2
6     0.375      (M1, M3)       2
7     0.500      (M1, M4)       2
8     0.500      (M4, M2)       2
9     0.375      (M5, M2)       2
10    0.500      (M5, M4)       2
11    0.375  (M1, M4, M2)       3
12    0.375  (M4, M5, M2)       3


## Frequent Itemsets: Select Itemsets Based on Criteria 

Now we can select an itemset based on criteria (such as length, support).

In [6]:
# Select all frequent 2-itemsets
frequent_2_itemsets = frequent_itemsets[(frequent_itemsets['length'] == 2)]

print("\nFrequent 2-itemsets:")
print(frequent_2_itemsets)


# Select all frequent 2-itemsets with minimum support 0.5
frequent_2_itemsets = frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                                        (frequent_itemsets['support'] >= 0.5) ]

print("\nFrequent 2-itemsets with Minimum Support 0.5:")
print(frequent_2_itemsets)


Frequent 2-itemsets:
    support  itemsets  length
5     0.500  (M1, M2)       2
6     0.375  (M1, M3)       2
7     0.500  (M1, M4)       2
8     0.500  (M4, M2)       2
9     0.375  (M5, M2)       2
10    0.500  (M5, M4)       2

Frequent 2-itemsets with Minimum Support 0.5:
    support  itemsets  length
5       0.5  (M1, M2)       2
7       0.5  (M1, M4)       2
8       0.5  (M4, M2)       2
10      0.5  (M5, M4)       2


## Association Rules

We use Mlxtend API's "association_rules" function to generate association rules (as a DataFrame object) including the metrics 'score', 'confidence', and 'lift'. This function requires the transaction data to be passed as a one-hot encoded DataFrame object.

        association_rules(df, metric='confidence', min_threshold=0.8, support_only=False)



### Parameters

- df : pandas DataFrame

        pandas DataFrame of frequent itemsets with columns ['support', 'itemsets']

- metric : string (default: 'confidence')

        Metric to evaluate if a rule is of interest. Automatically set to 'support' if support_only=True. Otherwise, supported metrics are 'support', 'confidence', 'lift', 'leverage', and 'conviction.


In [7]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

#rules.head() # use this method if the number of rules is very large
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(M1),(M2),0.750,0.625,0.500,0.666667,1.066667,0.03125,1.125
1,(M2),(M1),0.625,0.750,0.500,0.800000,1.066667,0.03125,1.250
2,(M1),(M3),0.750,0.375,0.375,0.500000,1.333333,0.09375,1.250
3,(M3),(M1),0.375,0.750,0.375,1.000000,1.333333,0.09375,inf
4,(M1),(M4),0.750,0.750,0.500,0.666667,0.888889,-0.06250,0.750
5,(M4),(M1),0.750,0.750,0.500,0.666667,0.888889,-0.06250,0.750
6,(M4),(M2),0.750,0.625,0.500,0.666667,1.066667,0.03125,1.125
7,(M2),(M4),0.625,0.750,0.500,0.800000,1.066667,0.03125,1.250
8,(M5),(M2),0.500,0.625,0.375,0.750000,1.200000,0.06250,1.500
9,(M2),(M5),0.625,0.500,0.375,0.600000,1.200000,0.06250,1.250


## Association Rules: Selecting Results

We can select association rules based on criteria (antecedent length, confidece, lift, etc.).

To do this we add a new column to the "rules" DataFrame that stores the length of the antecedents.

In [8]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

#rules.head() # use this method if the number of rules is very large
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(M1),(M2),0.750,0.625,0.500,0.666667,1.066667,0.03125,1.125,1
1,(M2),(M1),0.625,0.750,0.500,0.800000,1.066667,0.03125,1.250,1
2,(M1),(M3),0.750,0.375,0.375,0.500000,1.333333,0.09375,1.250,1
3,(M3),(M1),0.375,0.750,0.375,1.000000,1.333333,0.09375,inf,1
4,(M1),(M4),0.750,0.750,0.500,0.666667,0.888889,-0.06250,0.750,1
5,(M4),(M1),0.750,0.750,0.500,0.666667,0.888889,-0.06250,0.750,1
6,(M4),(M2),0.750,0.625,0.500,0.666667,1.066667,0.03125,1.125,1
7,(M2),(M4),0.625,0.750,0.500,0.800000,1.066667,0.03125,1.250,1
8,(M5),(M2),0.500,0.625,0.375,0.750000,1.200000,0.06250,1.500,1
9,(M2),(M5),0.625,0.500,0.375,0.600000,1.200000,0.06250,1.250,1


## Association Rules: Select Association Rules Based on Criteria

Following example select association rules based on 3 criteria.
- Antecedent length
- Confidence
- Lift

In [9]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.65) &
       (rules['lift'] > 1.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
19,"(M2, M4)",(M5),0.500,0.50,0.375,0.75,1.500000,0.12500,2.0,2
20,"(M5, M2)",(M4),0.375,0.75,0.375,1.00,1.333333,0.09375,inf,2
